In [1]:
import requests

In [2]:
url_pelis = f"https://beta.adalab.es/resources/apis/pelis/pelis.json"

In [3]:
peticion = requests.get(url_pelis)

In [4]:
peticion.status_code

200

In [5]:
datos_peliculas = peticion.json()

In [6]:
datos_peliculas

[{'id': 1,
  'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 2,
  'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 3,
  'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 4,
  'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False,
  'subtitulos': ['es', 'en', 'fr']},
 {'id': 5,
  'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 6,
  'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 7,
  'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos

In [7]:
info_peliculas = []

for pelicula in datos_peliculas:
    info_peliculas.append({"titulo": pelicula.get("titulo"), "año": pelicula.get("año"), "duracion": pelicula.get("duracion"), "genero": pelicula.get("genero"), "adultos": pelicula.get("adultos")})

In [8]:
info_peliculas

[{'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False},
 {'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False},
 {'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True},
 {'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False},
 {'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False},
 {'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True},
 {'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos': False},
 {'titulo': 'The Matrix',
  'año': 1999,
  'duracion': 136,
  'genero': 'Ciencia ficción',
  'adultos': False},
 {'titulo': 'The Shawshank Redemption',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False},
 {'titulo': 'Interstella

In [9]:
import mysql.connector
from mysql.connector import errorcode

cnx = None
cur = None

try:
    cnx = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="AlumnaAdalab",
    )
    print("Conectada a MySQL.")

    cur = cnx.cursor()

    cur.execute("CREATE DATABASE IF NOT EXISTS peliculas_bd")
    cur.execute("USE peliculas_bd")

    cnx.commit()
    print("peliculas_bd lista y seleccionada.")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Usuario o contraseña incorrectos.")
    else:
        print("Error MySQL:", err)
        print("Código:", getattr(err, "errno", None))
        print("SQLSTATE:", getattr(err, "sqlstate", None))

finally:
    if cur is not None:
        cur.close()
    if cnx is not None and cnx.is_connected():
        cnx.close()
        print("Conexión cerrada.")


Conectada a MySQL.
peliculas_bd lista y seleccionada.
Conexión cerrada.


In [10]:
import mysql.connector

DB_NAME = "peliculas_bd"

cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AlumnaAdalab",
    database=DB_NAME
)

cursor = cnx.cursor()

cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
cursor.execute("DROP TABLE IF EXISTS subtitulos")
cursor.execute("DROP TABLE IF EXISTS peliculas")
cursor.execute("DROP TABLE IF EXISTS generos")
cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
cnx.commit()

# Tabla generos

cursor.execute("""
CREATE TABLE generos (
    id_genero INT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(50) NOT NULL UNIQUE
) ENGINE=InnoDB;
""")

# Tabla peliculas

cursor.execute("""
CREATE TABLE peliculas (
    id_pelicula INT PRIMARY KEY,
    titulo VARCHAR(150) NOT NULL,
    anio YEAR NOT NULL,
    duracion INT NOT NULL,
    adultos BOOLEAN NOT NULL,
    genero_id INT NOT NULL,
    CONSTRAINT fk_peliculas_generos
        FOREIGN KEY (genero_id) REFERENCES generos(id_genero)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
) ENGINE=InnoDB;
""")

# Tabla subtitulos
 
cursor.execute("""
CREATE TABLE subtitulos (
    id_subtitulo INT AUTO_INCREMENT PRIMARY KEY,
    pelicula_id INT NOT NULL,
    idioma VARCHAR(10) NOT NULL,
    CONSTRAINT fk_subtitulos_peliculas
        FOREIGN KEY (pelicula_id) REFERENCES peliculas(id_pelicula)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    CONSTRAINT uq_pelicula_idioma UNIQUE (pelicula_id, idioma)
) ENGINE=InnoDB;
""")

cnx.commit()
print("Tablas recreadas")


Tablas recreadas


In [11]:
# Insertar géneros
generos_unicos = sorted({p.get("genero") for p in datos_peliculas if p.get("genero")})

sql_genero = "INSERT IGNORE INTO generos (nombre) VALUES (%s)"
cursor.executemany(sql_genero, [(g,) for g in generos_unicos])
cnx.commit()

# Id_genero 
cursor.execute("SELECT id_genero, nombre FROM generos")
map_generos = {nombre: id_genero for (id_genero, nombre) in cursor.fetchall()}

# Insertar películas 
sql_peli = """
INSERT INTO peliculas (id_pelicula, titulo, anio, duracion, adultos, genero_id)
VALUES (%s, %s, %s, %s, %s, %s)
"""

val_pelis = []
for p in datos_peliculas:
    val_pelis.append((
        p["id"],
        p["titulo"],
        p["año"],
        p["duracion"],
        p["adultos"],
        map_generos[p["genero"]]
    ))

cursor.executemany(sql_peli, val_pelis)
cnx.commit()

# Subtítulos
sql_subs = """
INSERT IGNORE INTO subtitulos (pelicula_id, idioma)
VALUES (%s, %s)
"""

val_subs = []
for pelicula in datos_peliculas:
    pelicula_id = pelicula["id"]
    for idioma in pelicula.get("subtitulos", []):
        val_subs.append((pelicula_id, idioma))

cursor.executemany(sql_subs, val_subs)
cnx.commit()

print("Datos insertados en generos, peliculas y subtitulos")


Datos insertados en generos, peliculas y subtitulos


In [12]:
cursor.execute("SELECT COUNT(*) FROM peliculas")
print("Películas:", cursor.fetchone()[0])



Películas: 100


In [13]:
#1. ¿Cuántas películas tienen una duración superior a 120 minutos?

cursor.execute("""
SELECT COUNT(*)
FROM peliculas
WHERE duracion > 120
""")

cantidad = cursor.fetchone()[0]
print(cantidad)


59


In [14]:
print(cnx.is_connected())


True


In [15]:
#2. ¿Cuántas películas incluyen subtítulos en español?

cursor.execute("""
SELECT COUNT(DISTINCT pelicula_id)
FROM subtitulos
WHERE idioma = 'es'
""")

cantidad = cursor.fetchone()[0]
print(f"{cantidad}")

100


In [16]:
#3. ¿Cuántas películas tienen contenido adulto?

cursor.execute("""
SELECT COUNT(*)
FROM peliculas
WHERE adultos = 1
""")

cantidad = cursor.fetchone()[0]
print(cantidad)

47


In [17]:
try:
    cursor.close()
except:
    pass

try:
    cnx.close()
except:
    pass

# Reconecta
cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AlumnaAdalab",
    database="peliculas_bd"
)


cursor = cnx.cursor(buffered=True)

print("Reconectada:", cnx.is_connected())

Reconectada: True


In [18]:
#4. ¿Cuál es la película más antigua registrada en la base de datos?

cursor.execute("""SELECT titulo
FROM peliculas
ORDER BY anio ASC
LIMIT 1;
""")
resultado = cursor.fetchone()[0]
print(resultado)


Citizen Kane


In [19]:
try:
    cursor.fetchall()
except:
    pass


In [20]:
#5. Muestra el promedio de duración de las películas agrupado por género.

cursor.execute("""
SELECT g.nombre, ROUND(AVG(p.duracion))
FROM generos AS g
LEFT JOIN peliculas AS p
  ON p.genero_id = g.id_genero
GROUP BY g.nombre
""")

for genero, promedio in cursor.fetchall():
    print(genero, promedio)


Acción 139
Animación 103
Aventura 133
Bélico 161
Biografía 142
Ciencia ficción 136
Comedia 117
Crimen 154
Drama 126
Fantasía 160
Musical 128
Romance 140
Suspense 120
Terror 117
Thriller 122
Western 167


In [21]:
#6. ¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.
cursor.execute("""
SELECT anio, COUNT(titulo)
FROM peliculas
GROUP BY anio
ORDER BY anio DESC
""")

resultados = cursor.fetchall()

for anio, cantidad in resultados:
    print(f"{anio}: {cantidad}")


2021: 1
2019: 3
2018: 3
2017: 4
2016: 3
2015: 3
2014: 3
2013: 4
2012: 3
2011: 1
2010: 3
2009: 3
2008: 4
2007: 1
2006: 2
2004: 3
2003: 3
2002: 3
2001: 5
2000: 3
1999: 4
1998: 3
1997: 3
1995: 2
1994: 4
1993: 2
1991: 2
1990: 1
1986: 1
1985: 1
1984: 1
1983: 1
1982: 1
1980: 3
1979: 1
1977: 1
1976: 2
1974: 1
1973: 1
1972: 1
1960: 1
1958: 1
1957: 1
1954: 1
1942: 1
1941: 1


In [22]:
#7. ¿Cuál es el año con más películas en la base de datos
cursor.execute("""
SELECT anio, COUNT(*) AS total_peliculas
FROM peliculas
GROUP BY anio
ORDER BY total_peliculas DESC
LIMIT 1
""")

anio, total = cursor.fetchone()
print(f" {anio} {total} ")



 2001 5 


In [23]:
#8. Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

cursor.execute("""
SELECT g.nombre, COUNT(p.id_pelicula) AS total
FROM generos AS g
LEFT JOIN peliculas AS p ON g.id_genero = p.genero_id
GROUP BY g.nombre
""")

resultados = cursor.fetchall()

for genero, total in resultados:
    print(f"{genero}: {total}")




Acción: 9
Animación: 9
Aventura: 3
Bélico: 2
Biografía: 3
Ciencia ficción: 13
Comedia: 1
Crimen: 7
Drama: 27
Fantasía: 5
Musical: 1
Romance: 3
Suspense: 2
Terror: 7
Thriller: 6
Western: 2


In [24]:
#9. Muestra todas las películas cuyo título contenga la palabra "Godfather" (puedes usar cualquier palabra).

cursor.execute("""
SELECT titulo
FROM peliculas
WHERE titulo LIKE '%Godfather%'
""")

for (titulo,) in cursor.fetchall():
    print(titulo)


The Godfather
The Godfather Part II
